In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv

angin571.csv  angin711.csv  angin714.csv  angin717.csv
angin572.csv  angin712.csv  angin715.csv  angin718.csv
angin573.csv  angin713.csv  angin716.csv


In [ ]:
%cd /content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv

/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv


In [ ]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import math
import scipy.stats as stats 
from numpy import array
from pandas import read_csv
from math import sqrt
from numpy import split
from plotly.subplots import make_subplots
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from datetime import datetime
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Pre-processing magnitude wind value

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin571.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          100.5       5.0  2022-01-01 00:00:00  1.193005
1          100.5       5.0  2022-01-01 01:00:00  1.251592
2          100.5       5.0  2022-01-01 02:00:00  1.124508
3          100.5       5.0  2022-01-01 03:00:00  0.941326
4          100.5       5.0  2022-01-01 04:00:00  0.598961
...          ...       ...                  ...       ...
14515      100.5       5.0  2023-04-30 21:00:00  2.910614
14516      100.5       5.0  2023-04-30 22:00:00  2.630344
14517      100.5       5.0  2023-04-30 22:00:00  2.350074
14518      100.5       5.0  2023-04-30 23:00:00  1.999639
14519      100.5       5.0  2023-04-30 23:00:00  1.649204

[14520 rows x 4 columns]


## Visualisasi Data Mentah

In [ ]:
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100

# Print the result
print("Percentage of NaN values in the wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the wind data.")
else:
    print("No NaN values found in the wind data.")
print(data)

Percentage of NaN values in the wind data: 0.00%
No NaN values found in the wind data.
       longitude  latitude                 time       mag
0          100.5       5.0  2022-01-01 00:00:00  1.193005
1          100.5       5.0  2022-01-01 01:00:00  1.251592
2          100.5       5.0  2022-01-01 02:00:00  1.124508
3          100.5       5.0  2022-01-01 03:00:00  0.941326
4          100.5       5.0  2022-01-01 04:00:00  0.598961
...          ...       ...                  ...       ...
14515      100.5       5.0  2023-04-30 21:00:00  2.910614
14516      100.5       5.0  2023-04-30 22:00:00  2.630344
14517      100.5       5.0  2023-04-30 22:00:00  2.350074
14518      100.5       5.0  2023-04-30 23:00:00  1.999639
14519      100.5       5.0  2023-04-30 23:00:00  1.649204

[14520 rows x 4 columns]


In [ ]:

# Menampilkan grafik
plt.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=data['time'], y=data['mag'],
                         mode='lines', name='Wind Data'))

# Edit the layout
fig.update_layout(xaxis_title='<b>Date</b>', yaxis_title='<b><i>Wind</i> [m/s]</b>')

fig.show()

## Simple Model

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.metrics import Accuracy

# Load and preprocess the data
swh = data["mag"].values.reshape(-1, 1)  # Reshape to a column vector
scaler = MinMaxScaler()
swh_scaled = scaler.fit_transform(swh)

# Split the data into training and testing sets
train_size = int(len(swh_scaled) * 0.8)
train_data = swh_scaled[:train_size]
test_data = swh_scaled[train_size:]

# Define the function to create the input sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)

# Set the sequence length and create the input sequences
seq_length = 10  # Adjust as needed
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(seq_length, 1)),
    tf.keras.layers.Dense(1)
])

# Create an instance of the Accuracy metric
# accuracy_metric = Accuracy()

# Compile the model and add the accuracy metric
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

# Evaluate the model
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)

# Make predictions
predictions = model.predict(X_test)

# Inverse scale the predictions and actual values
predictions = scaler.inverse_transform(predictions)
actual_values = scaler.inverse_transform(y_test)



Epoch 1/10
363/363 - 6s - loss: 0.6004 - accuracy: 8.6162e-05 - 6s/epoch - 17ms/step
Epoch 2/10
363/363 - 6s - loss: 0.5648 - accuracy: 8.6162e-05 - 6s/epoch - 17ms/step
Epoch 3/10
363/363 - 5s - loss: 0.5580 - accuracy: 1.7232e-04 - 5s/epoch - 13ms/step
Epoch 4/10
363/363 - 5s - loss: 0.5528 - accuracy: 1.7232e-04 - 5s/epoch - 14ms/step
Epoch 5/10
363/363 - 6s - loss: 0.5502 - accuracy: 1.7232e-04 - 6s/epoch - 17ms/step
Epoch 6/10
363/363 - 5s - loss: 0.5480 - accuracy: 1.7232e-04 - 5s/epoch - 13ms/step
Epoch 7/10
363/363 - 5s - loss: 0.5469 - accuracy: 1.7232e-04 - 5s/epoch - 15ms/step
Epoch 8/10
363/363 - 9s - loss: 0.5468 - accuracy: 1.7232e-04 - 9s/epoch - 25ms/step
Epoch 9/10
363/363 - 4s - loss: 0.5463 - accuracy: 1.7232e-04 - 4s/epoch - 12ms/step
Epoch 10/10
363/363 - 5s - loss: 0.5461 - accuracy: 1.7232e-04 - 5s/epoch - 15ms/step
91/91 [==============================] - 1s 4ms/step


In [ ]:
# Calculate RMSE
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('rmse:',rmse)
# print("Accuracy: {:.2f}%".format(accuracy * 100))
# Print RMSE
# print("RMSE: {:.2f}".format(rmse))
# # Print the predictions and actual values
# for i in range(len(predictions)):
#     print("Predicted:", predictions[i][0], "Actual:", actual_values[i][0])

91/91 [==============================] - 0s 4ms/step
rmse: 0.04294040668200857


In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the predicted values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test))),
    y=y_pred.flatten(),
    mode='lines',
    name='Predicted Values'
))

# Add the actual values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test))),
    y=y_test.flatten(),
    mode='lines',
    name='Actual Values'
))

# Set the layout
fig.update_layout(
    title='Predicted Values vs Actual Values',
    xaxis_title='Time',
    yaxis_title='Value',
)

# Show the figure
fig.show()


## Complex Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import plotly.graph_objs as go
import plotly.io as pio

# Load the data
# data = pd.read_csv('swh_data.csv')

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['mag'].values.reshape(-1, 1))

# Split the data into train and test sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Function to create sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Set the sequence length
seq_length = 60

# Create the training sequences
X_train, y_train = create_sequences(train_data, seq_length)

# Create the test sequences
X_test, y_test = create_sequences(test_data, seq_length)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=2)





Epoch 1/10
362/362 - 75s - loss: 0.5893 - accuracy: 8.6535e-05 - val_loss: 0.5721 - val_accuracy: 0.0000e+00 - 75s/epoch - 207ms/step
Epoch 2/10
362/362 - 48s - loss: 0.5733 - accuracy: 1.7307e-04 - val_loss: 0.5583 - val_accuracy: 0.0000e+00 - 48s/epoch - 133ms/step
Epoch 3/10
362/362 - 42s - loss: 0.5626 - accuracy: 1.7307e-04 - val_loss: 0.5451 - val_accuracy: 0.0000e+00 - 42s/epoch - 115ms/step
Epoch 4/10


KeyboardInterrupt: ignored

In [ ]:
# Generate predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse scale the predictions
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print(f"Train RMSE: {train_rmse:.2f}")
print(f"Test RMSE: {test_rmse:.2f}")

In [ ]:
# Create a figure for visualization
fig = go.Figure()

# Add the actual values trace
fig.add_trace(go.Scatter(
    x=data.index[:train_size],
    y=data['mag'][:train_size],
    name='Actual (Train)',
    mode='lines',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=data.index[train_size:],
    y=data['mag'][train_size:],
    name='Actual (Test)',
    mode='lines',
    line=dict(color='green')
))

# Add the predicted values trace
fig.add_trace(go.Scatter(
    x=data.index[seq_length:seq_length+len(train_predictions)],
    y=train_predictions.flatten(),
    name='Predicted (Train)',
    mode='lines',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=data.index[seq_length+train_size:seq_length+train_size+len(test_predictions)],
    y=test_predictions.flatten(),
    name='Predicted (Test)',
    mode='lines',
    line=dict(color='orange')
))

# Set the layout
fig.update_layout(
    title='Precipitation Prediction',
    xaxis_title='Date',
    yaxis_title='Precipitation',
    legend=dict(x=0, y=1),
)

# Save and display the figure
pio.show(fig)

# Fungsi


## splitting data

In [ ]:
# convert data into x and y 
def change_data(train, n_input, n_out=3):
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	for _ in range(n_input, len(data)):
		in_end = in_start + n_input
		out_end = in_end + n_out
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		in_start += 1
	return array(X), array(y)

In [ ]:
def splitting_data(data):
  # end_data = list(data['time']).index(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S' ))
  data= data.loc[:,['mag']]
  data= data.reset_index(drop=True)
 
  # scaler_label = StandardScaler().fit(data['swh'].values.reshape(-1, 1))
  # scaler_label = scaler_label.transform(data['swh'].values.reshape(-1, 1))

  scaler_label = StandardScaler().fit(data['mag'].values.reshape(-1, 1))
  scaler_label = scaler_label.transform(data['mag'].values.reshape(-1, 1))

  print(scaler_label)
  data_new = pd.DataFrame(scaler_label,index = data.index, columns=data.columns)
  train = data_new[:round(len(data_new)*0.8)]
  val = data_new[round(len(data_new)*0.8) : round(len(data_new)*0.9) ]
  test = data_new[round(len(data_new)*0.9):]

  i = 0
  while len(train) % 3 != 0:
    train = train[:round(len(train)*0.8) + i]
    i += 1
  j = 0
  while len(test) % 3 != 0:
    test = test[round(len(test)*0.9) - j:]
    j += 1
  
  val = val[round(len(val)*0.8)+i : round(len(val)*0.9)-j]
  while len(val) % 3 != 0:
    i -= 1
    val = val[round(len(val)*0.8)+i : round(len(val)*0.9)-j]


  # resturvture into windows of 12 hours data
  train = array(split(train, round(len(train)/3)))
  val = array(split(val, round(len(val)/3)))
  test = array(split(test, round(len(test)/3)))

  n_input,n_out = 9,3

  n_steps_in, n_steps_out = 9, 3
  X_train, y_train =  change_data(train,n_steps_in, n_steps_out)
  X_val, y_val =  change_data(val,n_steps_in, n_steps_out)

  n_features = 1
  return X_train, y_train, X_val, y_val, train, val, test ,j

## model 

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True    

def define_model(unit1,unit2):
  tf.keras.backend.clear_session()
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
  model1 = Sequential()
  model1.add(LSTM(unit1, activation='relu', input_shape=(n_timesteps, n_features)))
  model1.add((Dense(unit2, activation='relu')))
  model1.add((Dense(n_outputs)))

  # cp1 = ModelCheckpoint('model1/', save_best_only=True)
  model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), 
                 metrics=['accuracy'])
    
  return model1

def fit_model(X_train, y_train, X_val, y_val):
  # Instantiate the callback
  callbacks = myCallback()
  
  # fitting model 
  history = model1.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=10, callbacks=callbacks,verbose=2)
  # EXERCISE: Save the trained model as a Keras HDF5 file. 
  # saved_model_path = '/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/o2/model/'+daerah+'.h5'
  # YOUR CODE HERE
  
  # model1.save(saved_model_path)
  return history

In [ ]:
def forecast(data,daerah):

  scaler_label = StandardScaler().fit(data['mag'].values.reshape(-1, 1))
  # scaler_label = scaler_label.transform(data['chl'].values.reshape(-1, 1))
  data_new = data

  history = [x for x in val]
  predictions = list()
  n_input = 9
  for k in range(len(test)):
    datax = array(history)
    datax = datax.reshape((datax.shape[0]*datax.shape[1], datax.shape[2]))
    input_x = datax[-n_input:, :]
    input_x = input_x.reshape((1, len(input_x), 1))
    # predict the week
    yhat = model1.predict(input_x, verbose=0)
    yhat_sequence = yhat[0]
    # store the predictions
    predictions.append(yhat_sequence)
    # get real observation and add to history
    history.append(test[k, :])

  prediksi = array(predictions)
  prediksi = scaler_label.inverse_transform(predictions)
  
  observasi = test
  
  observasi = test[:, :, 0]
  observasi = scaler_label.inverse_transform(observasi)
  
  prediksi = prediksi.reshape((prediksi.shape[0]*prediksi.shape[1], 1))
  observasi = observasi.reshape((observasi.shape[0]*observasi.shape[1], 1))
  print(len(prediksi))
  print(len(observasi))
  print(prediksi.shape, observasi.shape)
  test_results = pd.DataFrame(data={'Predictions':prediksi.flatten(), 
                                   'Actuals':observasi.flatten()})
  
  test_results['Date'] = data_new['time'].iloc[round(len(data_new)*0.9) - j:].values
  # test_results.to_csv('/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/chl/prediksi/'+daerah+'.csv')
  return test_results
  
def forecast_viz():
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Predictions'],
                    mode='lines',
                    name='Predictions'))
  fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Actuals'],
                    mode='lines',
                    name='Observation'))
  
  # Edit the layout
  fig.update_layout( xaxis_title='<b>Date</b>',
                   yaxis_title='<b><i>Wind Magnitude</i> [m/s]</b>')
  
  return fig.show()

## Evaluating Model

In [ ]:
def evaluate():
  MSE = mean_squared_error(test_results['Actuals'],test_results['Predictions'])
  RMSE = math.sqrt(MSE)
  corr, _ = pearsonr(test_results['Actuals'],test_results['Predictions'])

  gap_sum = 0
  for j in range(len(test_results['Actuals'])):
    gap = test_results['Predictions'][j] - test_results['Actuals'][j]
    gap_sum += gap
    
  bias = gap_sum/len(test_results['Actuals'])
  _, pvalue = stats.ttest_ind(test_results['Actuals'],test_results['Predictions'], equal_var=True)

  value = [round(RMSE,5),round(corr,5), round(bias,5), round(pvalue,5)]
  eval = pd.DataFrame(value, index =['rmse', 'corr', 'bias','pvalue'],
                                              columns =['values'])
  return eval

# WPP571

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin571.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          100.5       5.0  2022-01-01 00:00:00  1.193005
1          100.5       5.0  2022-01-01 01:00:00  1.251592
2          100.5       5.0  2022-01-01 02:00:00  1.124508
3          100.5       5.0  2022-01-01 03:00:00  0.941326
4          100.5       5.0  2022-01-01 04:00:00  0.598961
...          ...       ...                  ...       ...
14515      100.5       5.0  2023-04-30 21:00:00  2.910614
14516      100.5       5.0  2023-04-30 22:00:00  2.630344
14517      100.5       5.0  2023-04-30 22:00:00  2.350074
14518      100.5       5.0  2023-04-30 23:00:00  1.999639
14519      100.5       5.0  2023-04-30 23:00:00  1.649204

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          100.5       5.0  2022-01-01 00:00:00  1.193005
1          100.5       5.0  2022-01-01 01:00:00  1.251592
2          100.5       5.0  2022-01-01 02:00:00  1.124508
3          100.5       5.0  2022-01-01 03:00:00  0.941326
4          100.5       5.0  2022-01-01 04:00:00  0.598961
...          ...       ...                  ...       ...
14515      100.5       5.0  2023-04-30 21:00:00  2.910614
14516      100.5       5.0  2023-04-30 22:00:00  2.630344
14517      100.5       5.0  2023-04-30 22:00:00  2.350074
14518      100.5       5.0  2023-04-30 23:00:00  1.999639
14519      100.5       5.0  2023-04-30 23:00:00  1.649204

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.70578419]
 [-0.64330265]
 [-0.77883438]
 ...
 [ 0.52820221]
 [ 0.15447163]
 [-0.21925896]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 14s - loss: 0.5217 - accuracy: 0.5200 - val_loss: 0.0864 - val_accuracy: 0.7500 - 14s/epoch - 38ms/step
Epoch 2/10
363/363 - 11s - loss: 0.4566 - accuracy: 0.5478 - val_loss: 0.2856 - val_accuracy: 0.7500 - 11s/epoch - 30ms/step
Epoch 3/10
363/363 - 11s - loss: 0.4521 - accuracy: 0.5474 - val_loss: 0.0521 - val_accuracy: 0.7500 - 11s/epoch - 30ms/step
Epoch 4/10
363/363 - 12s - loss: 0.4497 - accuracy: 0.5477 - val_loss: 0.1964 - val_accuracy: 0.7500 - 12s/epoch - 32ms/step
Epoch 5/10
363/363 - 9s - loss: 0.4468 - accuracy: 0.5473 - val_loss: 0.1827 - val_accuracy: 0.7500 - 9s/epoch - 26ms/step
Epoch 6/10
363/363 - 11s - loss: 0.4446 - accuracy: 0.5533 - val_loss: 0.0928 - val_accuracy: 0.7500 - 11s/epoch - 30ms/step
Epoch 7/10
363/363 - 11s - loss: 0.4429 - accuracy: 0.5527 - val_loss: 0.2087 - val_accuracy: 0.7500 - 11s/epoch - 30ms/step
Epoch 8/10
363/363 - 11s - loss: 0.4421 - accuracy: 0.5476 - val_loss: 0.0628 - val_accuracy: 0.7500 - 11s/epoch - 31ms/step
Ep

In [ ]:
test_results = forecast(data,'WPP571')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,2.652527,1.672709,2023-03-31 18:00:00
1,2.550624,1.746469,2023-03-31 18:00:00
2,2.447847,1.820228,2023-03-31 19:00:00
3,1.789088,1.893988,2023-03-31 19:00:00
4,1.664132,1.989812,2023-03-31 20:00:00
...,...,...,...
1447,2.381135,2.910614,2023-04-30 21:00:00
1448,2.257195,2.630344,2023-04-30 22:00:00
1449,2.338695,2.350074,2023-04-30 22:00:00
1450,2.120645,1.999639,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.50292
corr,0.83353
bias,0.04978
pvalue,0.08009


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp571'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP572

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin572.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          100.0      -4.0  2022-01-01 00:00:00  2.350570
1          100.0      -4.0  2022-01-01 01:00:00  2.501365
2          100.0      -4.0  2022-01-01 02:00:00  2.381148
3          100.0      -4.0  2022-01-01 03:00:00  2.758052
4          100.0      -4.0  2022-01-01 04:00:00  2.644835
...          ...       ...                  ...       ...
14515      100.0      -4.0  2023-04-30 21:00:00  0.667100
14516      100.0      -4.0  2023-04-30 22:00:00  0.931572
14517      100.0      -4.0  2023-04-30 22:00:00  1.196043
14518      100.0      -4.0  2023-04-30 23:00:00  1.404406
14519      100.0      -4.0  2023-04-30 23:00:00  1.612769

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          100.0      -4.0  2022-01-01 00:00:00  2.350570
1          100.0      -4.0  2022-01-01 01:00:00  2.501365
2          100.0      -4.0  2022-01-01 02:00:00  2.381148
3          100.0      -4.0  2022-01-01 03:00:00  2.758052
4          100.0      -4.0  2022-01-01 04:00:00  2.644835
...          ...       ...                  ...       ...
14515      100.0      -4.0  2023-04-30 21:00:00  0.667100
14516      100.0      -4.0  2023-04-30 22:00:00  0.931572
14517      100.0      -4.0  2023-04-30 22:00:00  1.196043
14518      100.0      -4.0  2023-04-30 23:00:00  1.404406
14519      100.0      -4.0  2023-04-30 23:00:00  1.612769

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.8240865 ]
 [-0.75413706]
 [-0.8099024 ]
 ...
 [-1.35964079]
 [-1.26298673]
 [-1.16633266]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 12s - loss: 0.2056 - accuracy: 0.3951 - val_loss: 0.0271 - val_accuracy: 0.7500 - 12s/epoch - 34ms/step
Epoch 2/10
363/363 - 10s - loss: 0.1023 - accuracy: 0.4316 - val_loss: 0.0175 - val_accuracy: 0.0000e+00 - 10s/epoch - 28ms/step
Epoch 3/10
363/363 - 11s - loss: 0.1022 - accuracy: 0.4348 - val_loss: 0.0367 - val_accuracy: 0.7500 - 11s/epoch - 30ms/step
Epoch 4/10
363/363 - 11s - loss: 0.1004 - accuracy: 0.4396 - val_loss: 0.0443 - val_accuracy: 0.0000e+00 - 11s/epoch - 30ms/step
Epoch 5/10
363/363 - 10s - loss: 0.1003 - accuracy: 0.4329 - val_loss: 0.0440 - val_accuracy: 0.0000e+00 - 10s/epoch - 28ms/step
Epoch 6/10
363/363 - 10s - loss: 0.0998 - accuracy: 0.4358 - val_loss: 0.0347 - val_accuracy: 0.0000e+00 - 10s/epoch - 28ms/step
Epoch 7/10
363/363 - 11s - loss: 0.1001 - accuracy: 0.4297 - val_loss: 0.0531 - val_accuracy: 0.0000e+00 - 11s/epoch - 32ms/step
Epoch 8/10
363/363 - 11s - loss: 0.1001 - accuracy: 0.4257 - val_loss: 0.0517 - val_accuracy: 0.0000e+00 

In [ ]:
test_results = forecast(data,'WPP572')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,5.202815,2.690965,2023-03-31 18:00:00
1,5.189708,2.726976,2023-03-31 18:00:00
2,5.152488,2.762987,2023-03-31 19:00:00
3,2.548205,2.798998,2023-03-31 19:00:00
4,2.683197,2.836195,2023-03-31 20:00:00
...,...,...,...
1447,0.891390,0.667100,2023-04-30 21:00:00
1448,1.106645,0.931572,2023-04-30 22:00:00
1449,0.946341,1.196043,2023-04-30 22:00:00
1450,1.151082,1.404406,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.43058
corr,0.94679
bias,-0.09077
pvalue,0.05359


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp572'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP573

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin573.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          112.5      -9.5  2022-01-01 00:00:00  6.295842
1          112.5      -9.5  2022-01-01 01:00:00  6.404226
2          112.5      -9.5  2022-01-01 02:00:00  6.363764
3          112.5      -9.5  2022-01-01 03:00:00  6.359508
4          112.5      -9.5  2022-01-01 04:00:00  6.501209
...          ...       ...                  ...       ...
14515      112.5      -9.5  2023-04-30 21:00:00  6.638583
14516      112.5      -9.5  2023-04-30 22:00:00  7.084063
14517      112.5      -9.5  2023-04-30 22:00:00  7.529542
14518      112.5      -9.5  2023-04-30 23:00:00  7.664552
14519      112.5      -9.5  2023-04-30 23:00:00  7.799562

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          112.5      -9.5  2022-01-01 00:00:00  6.295842
1          112.5      -9.5  2022-01-01 01:00:00  6.404226
2          112.5      -9.5  2022-01-01 02:00:00  6.363764
3          112.5      -9.5  2022-01-01 03:00:00  6.359508
4          112.5      -9.5  2022-01-01 04:00:00  6.501209
...          ...       ...                  ...       ...
14515      112.5      -9.5  2023-04-30 21:00:00  6.638583
14516      112.5      -9.5  2023-04-30 22:00:00  7.084063
14517      112.5      -9.5  2023-04-30 22:00:00  7.529542
14518      112.5      -9.5  2023-04-30 23:00:00  7.664552
14519      112.5      -9.5  2023-04-30 23:00:00  7.799562

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[0.33509701]
 [0.3811272 ]
 [0.36394333]
 ...
 [0.85904375]
 [0.91638186]
 [0.97371997]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 14s - loss: 0.1208 - accuracy: 0.3885 - val_loss: 0.0099 - val_accuracy: 0.5000 - 14s/epoch - 39ms/step
Epoch 2/10
363/363 - 12s - loss: 0.0613 - accuracy: 0.4070 - val_loss: 0.0088 - val_accuracy: 0.5000 - 12s/epoch - 32ms/step
Epoch 3/10
363/363 - 11s - loss: 0.0616 - accuracy: 0.3993 - val_loss: 0.0286 - val_accuracy: 0.5000 - 11s/epoch - 31ms/step
Epoch 4/10
363/363 - 11s - loss: 0.0608 - accuracy: 0.4191 - val_loss: 0.0134 - val_accuracy: 0.5000 - 11s/epoch - 29ms/step
Epoch 5/10
363/363 - 12s - loss: 0.0604 - accuracy: 0.4106 - val_loss: 0.0213 - val_accuracy: 0.5000 - 12s/epoch - 33ms/step
Epoch 6/10
363/363 - 12s - loss: 0.0607 - accuracy: 0.4112 - val_loss: 0.0156 - val_accuracy: 0.2500 - 12s/epoch - 32ms/step
Epoch 7/10
363/363 - 12s - loss: 0.0595 - accuracy: 0.4224 - val_loss: 0.0189 - val_accuracy: 0.5000 - 12s/epoch - 32ms/step
Epoch 8/10
363/363 - 11s - loss: 0.0596 - accuracy: 0.4318 - val_loss: 0.0170 - val_accuracy: 0.5000 - 11s/epoch - 32ms/step


In [ ]:
test_results = forecast(data,'WPP573')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,6.436558,5.054639,2023-03-31 18:00:00
1,6.453693,5.029832,2023-03-31 18:00:00
2,6.494462,5.005026,2023-03-31 19:00:00
3,4.997605,4.980220,2023-03-31 19:00:00
4,5.063429,4.948223,2023-03-31 20:00:00
...,...,...,...
1447,6.567748,6.638583,2023-04-30 21:00:00
1448,6.609425,7.084063,2023-04-30 22:00:00
1449,7.386446,7.529542,2023-04-30 22:00:00
1450,7.437452,7.664552,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.36437
corr,0.98041
bias,0.05052
pvalue,0.45404


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp573'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP711

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin711.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          108.5      -0.5  2022-01-01 00:00:00  2.006397
1          108.5      -0.5  2022-01-01 01:00:00  2.895625
2          108.5      -0.5  2022-01-01 02:00:00  3.499226
3          108.5      -0.5  2022-01-01 03:00:00  3.723068
4          108.5      -0.5  2022-01-01 04:00:00  3.960644
...          ...       ...                  ...       ...
14515      108.5      -0.5  2023-04-30 21:00:00  2.874625
14516      108.5      -0.5  2023-04-30 22:00:00  2.590415
14517      108.5      -0.5  2023-04-30 22:00:00  2.306206
14518      108.5      -0.5  2023-04-30 23:00:00  2.119212
14519      108.5      -0.5  2023-04-30 23:00:00  1.932218

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          108.5      -0.5  2022-01-01 00:00:00  2.006397
1          108.5      -0.5  2022-01-01 01:00:00  2.895625
2          108.5      -0.5  2022-01-01 02:00:00  3.499226
3          108.5      -0.5  2022-01-01 03:00:00  3.723068
4          108.5      -0.5  2022-01-01 04:00:00  3.960644
...          ...       ...                  ...       ...
14515      108.5      -0.5  2023-04-30 21:00:00  2.874625
14516      108.5      -0.5  2023-04-30 22:00:00  2.590415
14517      108.5      -0.5  2023-04-30 22:00:00  2.306206
14518      108.5      -0.5  2023-04-30 23:00:00  2.119212
14519      108.5      -0.5  2023-04-30 23:00:00  1.932218

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.98044293]
 [-0.53408941]
 [-0.23110791]
 ...
 [-0.82995217]
 [-0.92381483]
 [-1.01767749]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 14s - loss: 0.2990 - accuracy: 0.4203 - val_loss: 0.0040 - val_accuracy: 0.2500 - 14s/epoch - 39ms/step
Epoch 2/10
363/363 - 12s - loss: 0.1302 - accuracy: 0.4465 - val_loss: 0.0034 - val_accuracy: 0.2500 - 12s/epoch - 32ms/step
Epoch 3/10
363/363 - 12s - loss: 0.1283 - accuracy: 0.4517 - val_loss: 0.0022 - val_accuracy: 0.2500 - 12s/epoch - 32ms/step
Epoch 4/10
363/363 - 11s - loss: 0.1283 - accuracy: 0.4627 - val_loss: 0.0029 - val_accuracy: 0.2500 - 11s/epoch - 31ms/step
Epoch 5/10
363/363 - 10s - loss: 0.1271 - accuracy: 0.4545 - val_loss: 0.0052 - val_accuracy: 0.2500 - 10s/epoch - 28ms/step
Epoch 6/10
363/363 - 11s - loss: 0.1262 - accuracy: 0.4551 - val_loss: 0.0023 - val_accuracy: 0.2500 - 11s/epoch - 31ms/step
Epoch 7/10
363/363 - 11s - loss: 0.1259 - accuracy: 0.4484 - val_loss: 0.0043 - val_accuracy: 0.2500 - 11s/epoch - 31ms/step
Epoch 8/10
363/363 - 11s - loss: 0.1259 - accuracy: 0.4479 - val_loss: 0.0035 - val_accuracy: 0.2500 - 11s/epoch - 31ms/step


In [ ]:
test_results = forecast(data,'WPP711')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,2.530860,3.138732,2023-03-31 18:00:00
1,2.688090,3.258557,2023-03-31 18:00:00
2,2.783920,3.378381,2023-03-31 19:00:00
3,3.337646,3.498206,2023-03-31 19:00:00
4,3.411413,3.425376,2023-03-31 20:00:00
...,...,...,...
1447,2.998169,2.874625,2023-04-30 21:00:00
1448,3.028896,2.590415,2023-04-30 22:00:00
1449,2.250334,2.306206,2023-04-30 22:00:00
1450,2.350551,2.119212,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.49940
corr,0.92185
bias,0.05535
pvalue,0.22364


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp711'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP712

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin712.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          113.0      -6.0  2022-01-01 00:00:00  6.059773
1          113.0      -6.0  2022-01-01 01:00:00  5.611386
2          113.0      -6.0  2022-01-01 02:00:00  4.602724
3          113.0      -6.0  2022-01-01 03:00:00  4.159546
4          113.0      -6.0  2022-01-01 04:00:00  3.999988
...          ...       ...                  ...       ...
14515      113.0      -6.0  2023-04-30 21:00:00  1.876827
14516      113.0      -6.0  2023-04-30 22:00:00  2.078435
14517      113.0      -6.0  2023-04-30 22:00:00  2.280042
14518      113.0      -6.0  2023-04-30 23:00:00  2.315144
14519      113.0      -6.0  2023-04-30 23:00:00  2.350246

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          113.0      -6.0  2022-01-01 00:00:00  6.059773
1          113.0      -6.0  2022-01-01 01:00:00  5.611386
2          113.0      -6.0  2022-01-01 02:00:00  4.602724
3          113.0      -6.0  2022-01-01 03:00:00  4.159546
4          113.0      -6.0  2022-01-01 04:00:00  3.999988
...          ...       ...                  ...       ...
14515      113.0      -6.0  2023-04-30 21:00:00  1.876827
14516      113.0      -6.0  2023-04-30 22:00:00  2.078435
14517      113.0      -6.0  2023-04-30 22:00:00  2.280042
14518      113.0      -6.0  2023-04-30 23:00:00  2.315144
14519      113.0      -6.0  2023-04-30 23:00:00  2.350246

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 0.44548146]
 [ 0.27410401]
 [-0.11141498]
 ...
 [-0.99916367]
 [-0.98574728]
 [-0.97233089]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 14s - loss: 0.1505 - accuracy: 0.3444 - val_loss: 0.0131 - val_accuracy: 0.2500 - 14s/epoch - 37ms/step
Epoch 2/10
363/363 - 11s - loss: 0.0577 - accuracy: 0.3913 - val_loss: 0.0099 - val_accuracy: 0.7500 - 11s/epoch - 31ms/step
Epoch 3/10
363/363 - 11s - loss: 0.0563 - accuracy: 0.4100 - val_loss: 0.0084 - val_accuracy: 0.0000e+00 - 11s/epoch - 31ms/step
Epoch 4/10
363/363 - 11s - loss: 0.0551 - accuracy: 0.4092 - val_loss: 0.0236 - val_accuracy: 0.2500 - 11s/epoch - 30ms/step
Epoch 5/10
363/363 - 10s - loss: 0.0552 - accuracy: 0.4113 - val_loss: 0.0098 - val_accuracy: 0.2500 - 10s/epoch - 28ms/step
Epoch 6/10
363/363 - 11s - loss: 0.0551 - accuracy: 0.4078 - val_loss: 0.0196 - val_accuracy: 0.7500 - 11s/epoch - 29ms/step
Epoch 7/10
363/363 - 11s - loss: 0.0555 - accuracy: 0.4140 - val_loss: 0.0176 - val_accuracy: 0.2500 - 11s/epoch - 31ms/step
Epoch 8/10
363/363 - 11s - loss: 0.0551 - accuracy: 0.4193 - val_loss: 0.0230 - val_accuracy: 0.7500 - 11s/epoch - 31ms/s

In [ ]:
test_results = forecast(data,'WPP712')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp712'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

# WPP713

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin713.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
train.shape, val.shape, test.shape

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

In [ ]:
test_results = forecast(data,'WPP713')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,5.882302,3.239956,2023-03-31 18:00:00
1,5.958807,3.267469,2023-03-31 18:00:00
2,5.886674,3.294983,2023-03-31 19:00:00
3,3.568694,3.322496,2023-03-31 19:00:00
4,3.582889,3.920579,2023-03-31 20:00:00
...,...,...,...
1447,2.789259,2.767887,2023-04-30 21:00:00
1448,2.758128,3.137869,2023-04-30 22:00:00
1449,3.600805,3.507851,2023-04-30 22:00:00
1450,3.618606,3.776271,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.42768
corr,0.97414
bias,0.13480
pvalue,0.02392


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp713'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP714

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin714.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          127.5      -7.5  2022-01-01 00:00:00  5.116273
1          127.5      -7.5  2022-01-01 01:00:00  5.631596
2          127.5      -7.5  2022-01-01 02:00:00  6.160907
3          127.5      -7.5  2022-01-01 03:00:00  6.606712
4          127.5      -7.5  2022-01-01 04:00:00  7.085640
...          ...       ...                  ...       ...
14515      127.5      -7.5  2023-04-30 21:00:00  4.595117
14516      127.5      -7.5  2023-04-30 22:00:00  4.900418
14517      127.5      -7.5  2023-04-30 22:00:00  5.205718
14518      127.5      -7.5  2023-04-30 23:00:00  5.278656
14519      127.5      -7.5  2023-04-30 23:00:00  5.351594

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          127.5      -7.5  2022-01-01 00:00:00  5.116273
1          127.5      -7.5  2022-01-01 01:00:00  5.631596
2          127.5      -7.5  2022-01-01 02:00:00  6.160907
3          127.5      -7.5  2022-01-01 03:00:00  6.606712
4          127.5      -7.5  2022-01-01 04:00:00  7.085640
...          ...       ...                  ...       ...
14515      127.5      -7.5  2023-04-30 21:00:00  4.595117
14516      127.5      -7.5  2023-04-30 22:00:00  4.900418
14517      127.5      -7.5  2023-04-30 22:00:00  5.205718
14518      127.5      -7.5  2023-04-30 23:00:00  5.278656
14519      127.5      -7.5  2023-04-30 23:00:00  5.351594

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.19101379]
 [-0.01536583]
 [ 0.16504962]
 ...
 [-0.16052623]
 [-0.13566539]
 [-0.11080455]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 13s - loss: 0.1516 - accuracy: 0.3628 - val_loss: 0.0932 - val_accuracy: 0.5000 - 13s/epoch - 35ms/step
Epoch 2/10
363/363 - 9s - loss: 0.0427 - accuracy: 0.3704 - val_loss: 0.0221 - val_accuracy: 0.5000 - 9s/epoch - 24ms/step
Epoch 3/10
363/363 - 10s - loss: 0.0413 - accuracy: 0.3846 - val_loss: 0.0465 - val_accuracy: 0.5000 - 10s/epoch - 28ms/step
Epoch 4/10
363/363 - 10s - loss: 0.0405 - accuracy: 0.4059 - val_loss: 0.0661 - val_accuracy: 0.5000 - 10s/epoch - 29ms/step
Epoch 5/10
363/363 - 9s - loss: 0.0408 - accuracy: 0.4159 - val_loss: 0.0190 - val_accuracy: 0.5000 - 9s/epoch - 25ms/step
Epoch 6/10
363/363 - 10s - loss: 0.0393 - accuracy: 0.4325 - val_loss: 0.0393 - val_accuracy: 0.5000 - 10s/epoch - 27ms/step
Epoch 7/10
363/363 - 10s - loss: 0.0391 - accuracy: 0.4271 - val_loss: 0.0360 - val_accuracy: 0.5000 - 10s/epoch - 29ms/step
Epoch 8/10
363/363 - 10s - loss: 0.0392 - accuracy: 0.4289 - val_loss: 0.0663 - val_accuracy: 0.5000 - 10s/epoch - 27ms/step
Epoc

In [ ]:
test_results = forecast(data,'WPP714')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,1.091885,1.495605,2023-03-31 18:00:00
1,1.259357,1.646574,2023-03-31 18:00:00
2,1.432117,1.797543,2023-03-31 19:00:00
3,2.105312,1.948513,2023-03-31 19:00:00
4,2.147244,2.431722,2023-03-31 20:00:00
...,...,...,...
1447,4.461218,4.595117,2023-04-30 21:00:00
1448,4.354224,4.900418,2023-04-30 22:00:00
1449,5.088006,5.205718,2023-04-30 22:00:00
1450,5.034709,5.278656,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.38378
corr,0.98419
bias,-0.02842
pvalue,0.70876


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp714'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP715

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin715.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          127.0      -1.5  2022-01-01 00:00:00  3.674553
1          127.0      -1.5  2022-01-01 01:00:00  3.481419
2          127.0      -1.5  2022-01-01 02:00:00  3.550473
3          127.0      -1.5  2022-01-01 03:00:00  3.738927
4          127.0      -1.5  2022-01-01 04:00:00  4.448502
...          ...       ...                  ...       ...
14515      127.0      -1.5  2023-04-30 21:00:00  2.223209
14516      127.0      -1.5  2023-04-30 22:00:00  2.355126
14517      127.0      -1.5  2023-04-30 22:00:00  2.487043
14518      127.0      -1.5  2023-04-30 23:00:00  2.327336
14519      127.0      -1.5  2023-04-30 23:00:00  2.167629

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          127.0      -1.5  2022-01-01 00:00:00  3.674553
1          127.0      -1.5  2022-01-01 01:00:00  3.481419
2          127.0      -1.5  2022-01-01 02:00:00  3.550473
3          127.0      -1.5  2022-01-01 03:00:00  3.738927
4          127.0      -1.5  2022-01-01 04:00:00  4.448502
...          ...       ...                  ...       ...
14515      127.0      -1.5  2023-04-30 21:00:00  2.223209
14516      127.0      -1.5  2023-04-30 22:00:00  2.355126
14517      127.0      -1.5  2023-04-30 22:00:00  2.487043
14518      127.0      -1.5  2023-04-30 23:00:00  2.327336
14519      127.0      -1.5  2023-04-30 23:00:00  2.167629

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 0.00544141]
 [-0.09206108]
 [-0.05719951]
 ...
 [-0.5940639 ]
 [-0.6746907 ]
 [-0.7553175 ]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 12s - loss: 0.1923 - accuracy: 0.3853 - val_loss: 0.0402 - val_accuracy: 0.2500 - 12s/epoch - 34ms/step
Epoch 2/10
363/363 - 10s - loss: 0.1112 - accuracy: 0.4289 - val_loss: 0.0286 - val_accuracy: 0.7500 - 10s/epoch - 28ms/step
Epoch 3/10
363/363 - 9s - loss: 0.1082 - accuracy: 0.4519 - val_loss: 0.0120 - val_accuracy: 0.7500 - 9s/epoch - 25ms/step
Epoch 4/10
363/363 - 10s - loss: 0.1086 - accuracy: 0.4556 - val_loss: 0.0337 - val_accuracy: 0.7500 - 10s/epoch - 29ms/step
Epoch 5/10
363/363 - 10s - loss: 0.1077 - accuracy: 0.4601 - val_loss: 0.0037 - val_accuracy: 0.7500 - 10s/epoch - 29ms/step
Epoch 6/10
363/363 - 9s - loss: 0.1069 - accuracy: 0.4657 - val_loss: 0.0140 - val_accuracy: 0.7500 - 9s/epoch - 25ms/step
Epoch 7/10
363/363 - 10s - loss: 0.1078 - accuracy: 0.4551 - val_loss: 0.0052 - val_accuracy: 0.0000e+00 - 10s/epoch - 28ms/step
Epoch 8/10
363/363 - 10s - loss: 0.1064 - accuracy: 0.4591 - val_loss: 0.0058 - val_accuracy: 0.7500 - 10s/epoch - 29ms/step


In [ ]:
test_results = forecast(data,'WPP715')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,5.079169,1.334850,2023-03-31 18:00:00
1,5.088364,1.819146,2023-03-31 18:00:00
2,5.100898,2.303441,2023-03-31 19:00:00
3,2.330625,2.787737,2023-03-31 19:00:00
4,2.487207,3.487643,2023-03-31 20:00:00
...,...,...,...
1447,2.162574,2.223209,2023-04-30 21:00:00
1448,2.184951,2.355126,2023-04-30 22:00:00
1449,2.299694,2.487043,2023-04-30 22:00:00
1450,2.347975,2.327336,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.44058
corr,0.96887
bias,0.01663
pvalue,0.79958


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp715'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP716

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin716.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          125.0       1.5  2022-01-01 00:00:00  1.567107
1          125.0       1.5  2022-01-01 01:00:00  1.319858
2          125.0       1.5  2022-01-01 02:00:00  1.457125
3          125.0       1.5  2022-01-01 03:00:00  2.293795
4          125.0       1.5  2022-01-01 04:00:00  1.863349
...          ...       ...                  ...       ...
14515      125.0       1.5  2023-04-30 21:00:00  2.271244
14516      125.0       1.5  2023-04-30 22:00:00  2.205870
14517      125.0       1.5  2023-04-30 22:00:00  2.140496
14518      125.0       1.5  2023-04-30 23:00:00  2.099946
14519      125.0       1.5  2023-04-30 23:00:00  2.059396

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          125.0       1.5  2022-01-01 00:00:00  1.567107
1          125.0       1.5  2022-01-01 01:00:00  1.319858
2          125.0       1.5  2022-01-01 02:00:00  1.457125
3          125.0       1.5  2022-01-01 03:00:00  2.293795
4          125.0       1.5  2022-01-01 04:00:00  1.863349
...          ...       ...                  ...       ...
14515      125.0       1.5  2023-04-30 21:00:00  2.271244
14516      125.0       1.5  2023-04-30 22:00:00  2.205870
14517      125.0       1.5  2023-04-30 22:00:00  2.140496
14518      125.0       1.5  2023-04-30 23:00:00  2.099946
14519      125.0       1.5  2023-04-30 23:00:00  2.059396

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 0.06401786]
 [-0.22651622]
 [-0.06521867]
 ...
 [ 0.73779024]
 [ 0.690141  ]
 [ 0.64249176]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 13s - loss: 0.5396 - accuracy: 0.4142 - val_loss: 0.0770 - val_accuracy: 1.0000 - 13s/epoch - 35ms/step
Epoch 2/10
363/363 - 10s - loss: 0.1910 - accuracy: 0.4738 - val_loss: 0.0901 - val_accuracy: 1.0000 - 10s/epoch - 27ms/step
Epoch 3/10
363/363 - 10s - loss: 0.1843 - accuracy: 0.4750 - val_loss: 0.1041 - val_accuracy: 1.0000 - 10s/epoch - 27ms/step
Epoch 4/10
363/363 - 10s - loss: 0.1834 - accuracy: 0.4817 - val_loss: 0.1000 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoch 5/10
363/363 - 10s - loss: 0.1829 - accuracy: 0.4751 - val_loss: 0.1478 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoch 6/10
363/363 - 9s - loss: 0.1802 - accuracy: 0.4769 - val_loss: 0.0790 - val_accuracy: 1.0000 - 9s/epoch - 25ms/step
Epoch 7/10
363/363 - 10s - loss: 0.1808 - accuracy: 0.4776 - val_loss: 0.0951 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoch 8/10
363/363 - 11s - loss: 0.1789 - accuracy: 0.4882 - val_loss: 0.0369 - val_accuracy: 1.0000 - 11s/epoch - 29ms/step
Ep

In [ ]:
test_results = forecast(data,'WPP716')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,1.563458,1.594226,2023-03-31 18:00:00
1,1.523848,1.572402,2023-03-31 18:00:00
2,1.500757,1.550578,2023-03-31 19:00:00
3,1.453312,1.528753,2023-03-31 19:00:00
4,1.469511,1.582687,2023-03-31 20:00:00
...,...,...,...
1447,2.020963,2.271244,2023-04-30 21:00:00
1448,2.000081,2.205870,2023-04-30 22:00:00
1449,2.008016,2.140496,2023-04-30 22:00:00
1450,1.927917,2.099946,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.27089
corr,0.97112
bias,-0.04887
pvalue,0.18209


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp716'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP717

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin717.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          137.0       0.0  2022-01-01 00:00:00  1.659082
1          137.0       0.0  2022-01-01 01:00:00  2.772384
2          137.0       0.0  2022-01-01 02:00:00  2.705853
3          137.0       0.0  2022-01-01 03:00:00  1.611492
4          137.0       0.0  2022-01-01 04:00:00  1.153938
...          ...       ...                  ...       ...
14515      137.0       0.0  2023-04-30 21:00:00  3.407687
14516      137.0       0.0  2023-04-30 22:00:00  3.250479
14517      137.0       0.0  2023-04-30 22:00:00  3.093270
14518      137.0       0.0  2023-04-30 23:00:00  3.149258
14519      137.0       0.0  2023-04-30 23:00:00  3.205245

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          137.0       0.0  2022-01-01 00:00:00  1.659082
1          137.0       0.0  2022-01-01 01:00:00  2.772384
2          137.0       0.0  2022-01-01 02:00:00  2.705853
3          137.0       0.0  2022-01-01 03:00:00  1.611492
4          137.0       0.0  2022-01-01 04:00:00  1.153938
...          ...       ...                  ...       ...
14515      137.0       0.0  2023-04-30 21:00:00  3.407687
14516      137.0       0.0  2023-04-30 22:00:00  3.250479
14517      137.0       0.0  2023-04-30 22:00:00  3.093270
14518      137.0       0.0  2023-04-30 23:00:00  3.149258
14519      137.0       0.0  2023-04-30 23:00:00  3.205245

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-1.12201169]
 [-0.50415496]
 [-0.54107818]
 ...
 [-0.32607021]
 [-0.29499845]
 [-0.26392669]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 12s - loss: 0.2701 - accuracy: 0.4041 - val_loss: 0.0124 - val_accuracy: 1.0000 - 12s/epoch - 34ms/step
Epoch 2/10
363/363 - 10s - loss: 0.1334 - accuracy: 0.4557 - val_loss: 0.0129 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoch 3/10
363/363 - 10s - loss: 0.1318 - accuracy: 0.4496 - val_loss: 0.0036 - val_accuracy: 1.0000 - 10s/epoch - 27ms/step
Epoch 4/10
363/363 - 9s - loss: 0.1306 - accuracy: 0.4673 - val_loss: 0.0111 - val_accuracy: 1.0000 - 9s/epoch - 26ms/step
Epoch 5/10
363/363 - 10s - loss: 0.1290 - accuracy: 0.4576 - val_loss: 0.0110 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoch 6/10
363/363 - 10s - loss: 0.1292 - accuracy: 0.4583 - val_loss: 0.0058 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoch 7/10
363/363 - 9s - loss: 0.1286 - accuracy: 0.4539 - val_loss: 0.0186 - val_accuracy: 1.0000 - 9s/epoch - 25ms/step
Epoch 8/10
363/363 - 10s - loss: 0.1289 - accuracy: 0.4636 - val_loss: 0.0487 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoc

In [ ]:
test_results = forecast(data,'WPP717')
test_results

1452
1452
(1452, 1) (1452, 1)


,Predictions,Actuals,Date
0,2.017107,2.299285,2023-03-31 18:00:00
1,2.146271,2.206701,2023-03-31 18:00:00
2,2.150446,2.114117,2023-03-31 19:00:00
3,2.108257,2.021533,2023-03-31 19:00:00
4,2.161374,1.329151,2023-03-31 20:00:00
...,...,...,...
1447,3.344896,3.407687,2023-04-30 21:00:00
1448,3.201263,3.250479,2023-04-30 22:00:00
1449,3.097621,3.093270,2023-04-30 22:00:00
1450,3.062743,3.149258,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.42928
corr,0.98340
bias,0.05682
pvalue,0.48267


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp717'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 269,395
Trainable params: 269,395
Non-trainable params: 0
_________________________________________________________________


# WPP718

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/csv/angin718.csv')
data = data.sort_values('time')
data = data.reset_index(drop=True, inplace=False)
#calculate magnitude of wind
data['mag'] = data.apply(lambda x: math.sqrt(x['u10']**2 +x['v10']**2),axis=1)
#drop the u10, v10, and expver column
data.drop(['u10', 'v10', 'expver'], axis=1, inplace=True)
# print(data)

#interpolate nan values
data['mag'] = data['mag'].interpolate(method='linear', limit_direction='both')
print(data)

       longitude  latitude                 time       mag
0          136.5      -7.5  2022-01-01 00:00:00  5.148935
1          136.5      -7.5  2022-01-01 01:00:00  4.644215
2          136.5      -7.5  2022-01-01 02:00:00  3.958551
3          136.5      -7.5  2022-01-01 03:00:00  4.187444
4          136.5      -7.5  2022-01-01 04:00:00  4.417658
...          ...       ...                  ...       ...
14515      136.5      -7.5  2023-04-30 21:00:00  6.297655
14516      136.5      -7.5  2023-04-30 22:00:00  7.055602
14517      136.5      -7.5  2023-04-30 22:00:00  7.813549
14518      136.5      -7.5  2023-04-30 23:00:00  8.103739
14519      136.5      -7.5  2023-04-30 23:00:00  8.393929

[14520 rows x 4 columns]


### Interpolate the missing values

In [ ]:
#check the nan  values
nan_values = data['mag'].isna().any()
nan_percentage = (data['mag'].isna().sum() / len(data['mag'])) * 100
# print((data))
# Print the result
print("Percentage of NaN values in the Wind data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Wind data.")
else:
    print("No NaN values found in the Wind data.")
print(data)
# daerah = data

Percentage of NaN values in the Wind data: 0.00%
No NaN values found in the Wind data.
       longitude  latitude                 time       mag
0          136.5      -7.5  2022-01-01 00:00:00  5.148935
1          136.5      -7.5  2022-01-01 01:00:00  4.644215
2          136.5      -7.5  2022-01-01 02:00:00  3.958551
3          136.5      -7.5  2022-01-01 03:00:00  4.187444
4          136.5      -7.5  2022-01-01 04:00:00  4.417658
...          ...       ...                  ...       ...
14515      136.5      -7.5  2023-04-30 21:00:00  6.297655
14516      136.5      -7.5  2023-04-30 22:00:00  7.055602
14517      136.5      -7.5  2023-04-30 22:00:00  7.813549
14518      136.5      -7.5  2023-04-30 23:00:00  8.103739
14519      136.5      -7.5  2023-04-30 23:00:00  8.393929

[14520 rows x 4 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.0737939 ]
 [-0.25294614]
 [-0.49632494]
 ...
 [ 0.87201967]
 [ 0.97502362]
 [ 1.07802756]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model1 = define_model(256,20)
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 13s - loss: 0.1436 - accuracy: 0.3604 - val_loss: 0.2013 - val_accuracy: 1.0000 - 13s/epoch - 35ms/step
Epoch 2/10
363/363 - 10s - loss: 0.0580 - accuracy: 0.4039 - val_loss: 0.1085 - val_accuracy: 0.2500 - 10s/epoch - 29ms/step
Epoch 3/10
363/363 - 9s - loss: 0.0557 - accuracy: 0.4273 - val_loss: 0.0543 - val_accuracy: 0.0000e+00 - 9s/epoch - 25ms/step
Epoch 4/10
363/363 - 10s - loss: 0.0565 - accuracy: 0.4209 - val_loss: 0.1457 - val_accuracy: 0.0000e+00 - 10s/epoch - 28ms/step
Epoch 5/10
363/363 - 10s - loss: 0.0553 - accuracy: 0.4328 - val_loss: 0.1971 - val_accuracy: 0.0000e+00 - 10s/epoch - 29ms/step
Epoch 6/10
363/363 - 10s - loss: 0.0553 - accuracy: 0.4305 - val_loss: 0.1549 - val_accuracy: 0.2500 - 10s/epoch - 28ms/step
Epoch 7/10
363/363 - 9s - loss: 0.0551 - accuracy: 0.4499 - val_loss: 0.1059 - val_accuracy: 0.5000 - 9s/epoch - 25ms/step
Epoch 8/10
363/363 - 10s - loss: 0.0549 - accuracy: 0.4568 - val_loss: 0.0749 - val_accuracy: 0.0000e+00 - 10s/epoch 

In [ ]:
test_results = forecast(data,'WPP718')
test_results

In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/angin/model'+'modelanginwpp718'+'.h5'
model1.save(saved_model_path)

In [ ]:
mdl_new = load_model(saved_model_path)
mdl_new.summary()